In [111]:
import pandas_datareader.data as web
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

import seaborn as sns
sns.set_style('darkgrid')
sns.set_palette('viridis')

from datetime import datetime

from sklearn import linear_model 

In [89]:
tickers = list(pd.read_csv("tickers.csv").Ticker)
profiles = pd.read_csv("profiles.csv").set_index('symbol')
quotes = pd.read_csv("SP500.csv").set_index('symbol').drop("Unnamed: 0",axis =1)

# Data Inspection
First lets take a look at one stock before we format the whole lot. We'll choose Alphabet. Lets first take a look at the company profile:

In [159]:
tick = 'GOOG'
profiles.loc[tick]

Unnamed: 0                                                         204
price                                                          2092.91
beta                                                           1.02276
volAvg                                                     1.62444e+06
mktCap                                                   1407998890000
lastDiv                                                              0
range                                                1013.536-2123.547
changes                                                          -5.09
companyName                                              Alphabet Inc.
currency                                                           USD
cik                                                        1.65204e+06
isin                                                               NaN
cusip                                                              NaN
exchange                                          Nasdaq Global Select
exchan

next, we can load in the company's key metrics

In [160]:
df = pd.read_csv("company-key-metrics/SP500-{}-company-key-metrics.csv".format(tick)).drop("Unnamed: 0", axis=1)
df

,date,Revenue per Share,Net Income per Share,Operating Cash Flow per Share,Free Cash Flow per Share,Cash per Share,Book Value per Share,Tangible Book Value per Share,Shareholders Equity per Share,Interest Debt per Share,...,Average Payables,Average Inventory,Days Sales Outstanding,Days Payables Outstanding,Days of Inventory on Hand,Receivables Turnover,Payables Turnover,Inventory Turnover,ROE,Capex per Share
0,2019-12-31,233.715419,49.590000,78.724829,44.722403,26.710416,290.874670,365.764335,290.874670,6.720201,...,5.561000e+09,9.990000e+08,61.996577,28.231960,5.071701,5.887422,12.928610,71.967968,0.170486,-34.002426
1,2018-12-31,194.542753,43.703477,68.209901,32.464790,23.747130,252.569015,302.415095,252.569015,5.866754,...,4.969500e+09,1.053000e+09,56.537798,26.834540,6.785252,6.455858,13.601873,53.793135,0.173036,-35.745110
2,2017-12-31,157.557591,17.996430,52.717231,33.978885,15.229170,216.750237,252.785737,216.750237,5.796039,...,4.349000e+09,8.740000e+08,61.587885,25.119123,5.997521,5.926490,14.530762,60.858478,0.083028,-18.738345
3,2016-12-31,129.198833,27.877247,51.575341,36.959751,18.488463,198.990706,211.422258,198.990706,5.809310,...,3.801000e+09,6.335000e+08,57.544754,21.201121,2.783881,6.342889,17.216071,131.111940,0.140093,-14.615589
4,2015-12-31,108.220167,23.592571,38.347308,23.987993,23.882643,173.655348,184.354841,173.655348,5.915460,...,3.746000e+09,NaN,67.700396,25.025387,0.000000,5.391401,14.585189,NaN,0.135859,-14.359315


By inspection we can see we have data from the start of 2010 to the end of 2019. We can also get the column values using `df.columns`

In [161]:
df.columns

Index(['date', 'Revenue per Share', 'Net Income per Share',
       'Operating Cash Flow per Share', 'Free Cash Flow per Share',
       'Cash per Share', 'Book Value per Share',
       'Tangible Book Value per Share', 'Shareholders Equity per Share',
       'Interest Debt per Share', 'Market Cap', 'Enterprise Value', 'PE ratio',
       'Price to Sales Ratio', 'POCF ratio', 'PFCF ratio', 'PB ratio',
       'PTB ratio', 'EV to Sales', 'Enterprise Value over EBITDA',
       'EV to Operating cash flow', 'EV to Free cash flow', 'Earnings Yield',
       'Free Cash Flow Yield', 'Debt to Equity', 'Debt to Assets',
       'Net Debt to EBITDA', 'Current ratio', 'Interest Coverage',
       'Income Quality', 'Dividend Yield', 'Payout Ratio', 'SG&A to Revenue',
       'R&D to Revenue', 'Intangibles to Total Assets',
       'Capex to Operating Cash Flow', 'Capex to Revenue',
       'Capex to Depreciation', 'Stock-based compensation to Revenue',
       'Graham Number', 'ROIC', 'Return on Tangible Asse

we also have general quarterly data from the S&P500

### Calculating the SGR
straight away we can calulate the sustainable growth rate (SGR), which is given as:
>_**SGR** = Return on Equity * (1 – Dividend payout ratio)_

We can then average over the 10 years we get in the statement to get a more stable projection for growth

In [162]:
df['SGR'] = df.apply(lambda row: row.ROE*(1-row['Payout Ratio']),axis = 1)

In [163]:
SGR = df.SGR.mean()
SGR

0.14050036064315002

this gives an average growth rate of 14.1%

### Finding the risk-free rate of return
We can estimate the current risk free rate of return from US treasury 1 year T-bills

In [166]:
start = datetime(2019, 2, 1)
end =datetime(2021, 2, 1)

# get data from FED
Treasury = web.DataReader(['TB1YR'], 'fred', start, end)
RF = float(Treasury.iloc[-1])
RF = RF/100
print("CURRENT RISK FREE RATE: {}%".format(100*RF))

CURRENT RISK FREE RATE: 0.1%


the current rate is 0.1%, this is much much lower normal.

In [167]:
Treasury.T

DATE,2019-02-01,2019-03-01,2019-04-01,2019-05-01,2019-06-01,2019-07-01,2019-08-01,2019-09-01,2019-10-01,2019-11-01,...,2020-04-01,2020-05-01,2020-06-01,2020-07-01,2020-08-01,2020-09-01,2020-10-01,2020-11-01,2020-12-01,2021-01-01
TB1YR,2.47,2.41,2.34,2.27,1.94,1.91,1.73,1.75,1.57,1.53,...,0.18,0.16,0.18,0.15,0.13,0.13,0.13,0.12,0.1,0.1


### CAPM Cost of Equity
because not all companies pay dividends, we may want to use the Capital Asset Pricing Model (CAPM) for calculating the cost of equity. First we can get `beta`, which is  a measure of the volatility of a security compared to the market as a whole

In [171]:
beta = profiles.loc[tick].beta

once we have `beta` we'll need the yearly return for the market over the past year

In [169]:
start = datetime(2020, 2, 1)
end = datetime(2021, 2, 1) 
SP500 = web.DataReader(['sp500'], 'fred', start, end)
#Drop all Not a number values using drop method.
SP500.dropna(inplace = True)
SP500yearlyreturn = (SP500['sp500'].iloc[-1]/ SP500['sp500'].iloc[0])-1
print("YEARLY ROE FOR THE S&P500: {}".format(SP500yearlyreturn))

YEARLY ROE FOR THE S&P500: 0.1615736921807862


we can now work out the CAPM cost of equity

In [170]:
CAPM = RF+(beta*(SP500yearlyreturn - RF))
print("COST OF EQUITY FOR {}: {}".format(tick,CAPM))

COST OF EQUITY FOR GOOG: 0.1652291522832818


In [150]:
PPS = quotes.loc[tick].price
Div_yield = df.iloc[0]['Dividend Yield']
Div = PPS*Div_yield